# Automated Extraction of Quotations from Text - Model Deployment

The purpose is to develop an Application Programming Interface (API) from the selected model to predict and extract quotations from text. The model is loaded, extracting of quotes is tested, then the API is deployed using ngrok to be tested in the Postman app.

## Prepare API

### Load Libraries

In [ ]:
!pip install --quiet pyngrok flask_ngrok

Get your ngrok token at https://ngrok.com

In [ ]:
Authtoken = input('Authtoken: ')

Authtoken: Test


In [ ]:
!ngrok authtoken {Authtoken}

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras import preprocessing as kprocessing
from tensorflow.keras.models import load_model

import nltk
from nltk.tokenize import sent_tokenize

import pickle

from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, jsonify

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pd.options.display.max_colwidth = 100

### Load Data

In [ ]:
sentences_test = [
  'A journey of a thousand miles begins with a single step.',
  'Are you beginning a journey of a thousand steps tomorrow?',
  'The Post cited people familiar with the investigation as saying that federal agents were looking for classified documents related to nuclear weapons',
  'But it turns out that Trump did take such material from the White House.',
  'No one will reap except what they sow.'
]

In [ ]:
passage_carter = """Sam squinted against the sun at the distant dust trail raked up by the car on its way up to the Big House. The horses kicked and flicked their tails at flies, not caring about their owner's first visit in ten months. Sam waited. Mr Carter didn't come out here unless he had to, which was just fine by Sam. The more he kept out of his boss's way, the longer he'd have a job.
Carter came by later while Sam was chopping wood. Carter lifted his hat as if he were waiting for an appointment with the town priest, and then removed it completely as if he were talking to his mother. He pulled out a pile of paper from his back pocket and held it out.
'Don't pick up your mail often, do you?'
Sam took it without a glance and dropped the envelopes onto the bench.
'Never,' he replied and waited for Carter to say why he was here. The fact it was Carter's house was no explanation and they both knew it. Carter twisted his hat round and round, licking his lips and clearing his throat.
'Nice work fixing those fences,' he said finally.
'I'll be back to the beginning soon,' Sam said. It wasn't a complaint. A fence that took a year to repair meant another year's work to the man who did it well.
'Don't you ever want to take a holiday?'
'And go where?' A holiday meant being back out in the real world, a place even people like Carter travelled to escape from. Sam's escape was his reality and he wasn't going back.
Mr Carter wiped the sweat from the back of his neck. The damp patches on his shirt drew together like shapes in an atlas. His skin was already turning ruddy in the June sun. Otherwise he had the indoor tan of a man that made money while other people did the work.
'I've brought my son with me on this trip. He's had some trouble at school.' Mr Carter's eyes flicked up, blinked rapidly and then shifted back to the hat occupying his hands. 'Not much trouble out here for a young boy.' He attempted a laugh but it came out like a dog's bark.
The two men looked towards the northern end of the property. It stretched as far as the eye could see. Even the fences were barely visible from where they stood. However bored and rebellious a teenage boy might get, it wasn't possible to escape on foot. Sam looked at the biggest of the horses, kicking at the ground with its heavy hooves. Could the boy ride? he wondered. There was a whole load of trouble a good rider could get into out here, miles away from anyone. But maybe there was even more trouble for someone who knew nothing about horses and wanted to get away from his father."""

In [ ]:
passage_carnegie = """A man was warned by an officer for not putting his dog on a leash. Some days later, he was caught with the same offence. Rather than wait for the officer to start speaking, he took the initiative by admitting his mistake, “Officer, you’ve caught me red-handed. I’m guilty. I have no excuses.”  The chances are, when you begin to condemn yourself, the officer would want to feel important and nourish his self-esteem by showing you mercy.

During a course in human relations, a class wrote down criticisms to a certain man to let him see himself as others see him. One man was broken-hearted because he was denounced for being too sure of himself, too self-centered, too domineering, an egoist, trouble-maker, and a communist. One of his critics ordered him to get out of class. Instead of denouncing his critics, he said, “Boys, I certainly am unpopular. There can be no mistaking that. It hurts me to read these comments, but they are good for me. They have taught me a lesson. I long for friends just as you do. I want to make people like me. Won’t you help me? Won’t you please write me some more criticisms and tell me what I can do to improve my personality? If you will, I’ll try hard, awfully hard, to change.” Because of his soft answer and his sincerity, his words moved his critics – the very men who had denounced him one week earlier were now ready to support him."""

### Load Model

In [ ]:
!wget -q https://placeholder.com/rnn.h5
!wget -q https://placeholder.com/corpus.model

In [ ]:
model = load_model('rnn.h5')
corpus = pickle.load(open('corpus.model', 'rb'))

In [ ]:
max_words = 30000
max_len = 200

tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', num_words=max_words, oov_token="<pad>", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(corpus)

### Create Functions

In [ ]:
def sent_predict(sentences, tokenizer=tokenizer, threshold=0.5):
  seq = kprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(sentences), maxlen=max_len)
  predictions = model.predict(seq).reshape(-1)
  predictions = dict(zip(sentences, predictions))
  predictions = { key:float(value) for (key, value) in predictions.items() if value >= threshold }
  return predictions

In [ ]:
def extract_quotes(text, threshold=0.5):
  return sent_predict(sent_tokenize(text), threshold=threshold)

In [ ]:
def show_quotes(quotes):
  df = pd.DataFrame(quotes.items(), columns=['Sentence', 'Probability'])
  df.sort_values('Probability', ascending=False, ignore_index=True, inplace=True)
  return df

## Test Extraction

### Extract Quotes

In [ ]:
output_test = sent_predict(sentences_test)
output_test

{'A journey of a thousand miles begins with a single step.': 0.9970009326934814,
 'Are you beginning a journey of a thousand steps tomorrow?': 0.9919606447219849,
 'No one will reap except what they sow.': 0.9575396776199341}

In [ ]:
output_carter = extract_quotes(passage_carter)
output_carter

{'Sam squinted against the sun at the distant dust trail raked up by the car on its way up to the Big House.': 0.8071490526199341,
 "'Don't pick up your mail often, do you?'": 0.8967397212982178,
 "'Don't you ever want to take a holiday?'": 0.8882098197937012,
 'A holiday meant being back out in the real world, a place even people like Carter travelled to escape from.': 0.8992186784744263}

In [ ]:
output_carnegie = extract_quotes(passage_carnegie, threshold=0.5)
output_carnegie

{'I have no excuses.”  The chances are, when you begin to condemn yourself, the officer would want to feel important and nourish his self-esteem by showing you mercy.': 0.9042152166366577,
 'It hurts me to read these comments, but they are good for me.': 0.7593796253204346,
 'I long for friends just as you do.': 0.7520382404327393,
 'I want to make people like me.': 0.9578741788864136,
 'Won’t you please write me some more criticisms and tell me what I can do to improve my personality?': 0.9555157423019409,
 'If you will, I’ll try hard, awfully hard, to change.” Because of his soft answer and his sincerity, his words moved his critics – the very men who had denounced him one week earlier were now ready to support him.': 0.8130112886428833}

### Show Quotes

In [ ]:
show_quotes(output_test)

,Sentence,Probability
0,A journey of a thousand miles begins with a single step.,0.997001
1,Are you beginning a journey of a thousand steps tomorrow?,0.991961
2,No one will reap except what they sow.,0.957540


In [ ]:
show_quotes(output_carter)

,Sentence,Probability
0,"A holiday meant being back out in the real world, a place even people like Carter travelled to e...",0.899219
1,"'Don't pick up your mail often, do you?'",0.896740
2,'Don't you ever want to take a holiday?',0.888210
3,Sam squinted against the sun at the distant dust trail raked up by the car on its way up to the ...,0.807149


In [ ]:
show_quotes(output_carnegie)

,Sentence,Probability
0,I want to make people like me.,0.957874
1,Won’t you please write me some more criticisms and tell me what I can do to improve my personality?,0.955516
2,"I have no excuses.” The chances are, when you begin to condemn yourself, the officer would want...",0.904215
3,"If you will, I’ll try hard, awfully hard, to change.” Because of his soft answer and his sinceri...",0.813011
4,"It hurts me to read these comments, but they are good for me.",0.759380
5,I long for friends just as you do.,0.752038


## Deploy API

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def home():
  return jsonify(output_test)

In [ ]:
@app.route('/api', methods=['POST'])
def api():
  if request.form.get('data'):
    data = request.form['data']
    threshold = 0.5

    if request.form.get('threshold'):
      th = request.form['threshold']

      try:
        th = float(th)
        
        if th >= 0.0 and th <= 1.0:
          threshold = th

      except ValueError:
        pass

    quotes = extract_quotes(data, threshold=threshold)
    return quotes
  else:
    return 'Please include text in the POST request.', 401

In [ ]:
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://334a-35-194-85-139.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
